# **Data Cleaning**


## Objectives

- **Evaluate Missing Data**: Assess and address missing data within the dataset.
- **Clean Data**: Perform necessary data cleaning operations to enhance data quality.

## Inputs

- Dataset: `outputs/datasets/cleaned/house_price_records.csv`

## Outputs

- **Generate Cleaned Train and Test Sets**: The cleaned datasets for both training and testing will be saved under `outputs/datasets/cleaned`.

## Additional Information

- Data Imputation: Handle missing data in object-type columns.
- Feature Removal: Identify and remove unnecessary or redundant features to streamline the dataset.

The data cleaning process aims to improve data quality by addressing missing values and optimizing the dataset for further analysis and modeling. 'Time'].]. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/Spectral-data'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces'

# Load Collected data

In [4]:
import pandas as pd
df_raw_path = "/workspaces/Spectral-data/outputs/datasets/collection/spectra.csv"
df = pd.read_csv(df_raw_path)
df.head(3)

,sample,1,2,3,4,5,6
0,spectrum2.csv,20,30,40,50,60,70
1,spectrum1.csv,5,10,20,30,40,50


# Data Integration

In [5]:
integrated_df = df.copy()
integrated_df.iloc[:, 1:] = df.iloc[:, 1:].cumsum(axis=1)
integrated_df = integrated_df.iloc[:, [0, -1]]
integrated_df.rename(columns={"6": "int1"}, inplace=True) ## changing column name
integrated_df

,sample,int1
0,spectrum2.csv,270
1,spectrum1.csv,155


In [6]:
new_df = df.iloc[:, 3:7]
new_df.iloc[:, 1:] = new_df.iloc[:, 1:].cumsum(axis=1)
new_df

integrated_df['int2'] = new_df.iloc[:, -1]
integrated_df

,sample,int1,int2
0,spectrum2.csv,270,180
1,spectrum1.csv,155,120


## upgraded integration function

In [20]:
def calculate_integrals(dataframe, range_start, range_end, wavenumber1, wavenumber2):
    integrals = []

    for i in range(len(dataframe)):
        # Get the absorbances at the specified wavenumbers
        absorbance1 = dataframe.iloc[i, wavenumber1]
        absorbance2 = dataframe.iloc[i, wavenumber2]

        # Calculate the baseline as the average of the two absorbances
        baseline = (absorbance1 + absorbance2) / 2

        # Subtract the baseline from the spectrum within the specified wavenumber range
        spectrum = dataframe.iloc[i, range_start:range_end + 1] - baseline

        # Calculate the integral for the spectrum within the specified wavenumber range
        integral = spectrum.sum()
        integrals.append(integral)

    return integrals

# Create a list of settings with different integration ranges and baseline wavenumbers
integration_settings_list = [
    {'range_start': 3, 'range_end': 7, 'wavenumber1': 3, 'wavenumber2': 4, 'column_name': 'Integral_1'},
    {'range_start': 5, 'range_end': 9, 'wavenumber1': 5, 'wavenumber2': 6, 'column_name': 'Integral_2'},
    # Add more settings as needed
]

# Create a DataFrame to store all integration results
integration_df = df[['sample']].copy()

# Calculate and add integrals with different settings to the DataFrame
for settings in integration_settings_list:
    integrals = calculate_integrals(df, settings['range_start'], settings['range_end'], settings['wavenumber1'], settings['wavenumber2'])
    integration_df[settings['column_name']] = integrals

print(integration_df)

          sample  Integral_1  Integral_2
0  spectrum2.csv        10.0       -50.0
1  spectrum1.csv        27.5       -32.5


## Peakheight function

In [12]:
def calculate_peak_heights(dataframe, wavenumber1, wavenumber2):
    peak_heights = []

    for i in range(len(dataframe)):
        # Get the absorbances at the specified wavenumbers
        absorbance1 = dataframe.iloc[i, wavenumber1]
        absorbance2 = dataframe.iloc[i, wavenumber2]

        # Calculate the baseline as the average of the two absorbances
        baseline = (absorbance1 + absorbance2) / 2

        # Calculate the peak height for the spectrum based on the baseline
        peak_height = dataframe.iloc[i, range_start:range_end + 1].max() - baseline
        peak_heights.append(peak_height)

    return peak_heights

# Example usage with wavenumbers for baseline (e.g., 3 and 4)
wavenumber1 = 1  # Replace with the first wavenumber
wavenumber2 = 6  # Replace with the second wavenumber

range_start = 2  # Replace with the start of the range
range_end = 5    # Replace with the end of the range

# Create a new DataFrame to store the peak heights
df_peak_heights = df[['sample']].copy()
df_peak_heights['peak_height'] = peak_heights

print(df_peak_heights)

          sample  peak_height
0  spectrum2.csv         15.0
1  spectrum1.csv         12.5


## upgraded peakheight more setttings

In [21]:
# Assuming you have your spectral data loaded in 'df' as per your initial code

def calculate_peak_heights(dataframe, wavenumber1, wavenumber2, range_start, range_end):
    peak_heights = []

    for i in range(len(dataframe)):
        # Get the absorbances at the specified wavenumbers
        absorbance1 = dataframe.iloc[i, wavenumber1]
        absorbance2 = dataframe.iloc[i, wavenumber2]

        # Calculate the baseline as the average of the two absorbances
        baseline = (absorbance1 + absorbance2) / 2

        # Calculate the peak height for the spectrum within the specified wavenumber range
        peak_height = dataframe.iloc[i, range_start:range_end + 1].max() - baseline
        peak_heights.append(peak_height)

    return peak_heights

# Create a list of settings with different wavenumbers and ranges
settings_list = [
    {'wavenumber1': 3, 'wavenumber2': 4, 'range_start': 3, 'range_end': 7, 'column_name': 'peakheight_1'},
    {'wavenumber1': 5, 'wavenumber2': 6, 'range_start': 5, 'range_end': 9, 'column_name': 'Peakheight_2'},
    # Add more settings as needed
]

# Create a DataFrame to store all peak heights
peak_heights_df = df[['sample']].copy()

# Calculate and add peak heights with different settings to the DataFrame
for settings in settings_list:
    peak_heights = calculate_peak_heights(df, settings['wavenumber1'], settings['wavenumber2'], settings['range_start'], settings['range_end'])
    peak_heights_df[settings['column_name']] = peak_heights

print(peak_heights_df)


          sample  peakheight_1  Peakheight_2
0  spectrum2.csv          25.0           5.0
1  spectrum1.csv          25.0           5.0


## marging of calculation frames

In [23]:
merged_df = peak_heights_df.merge(integration_df, on='sample', how='inner')
merged_df

,sample,peakheight_1,Peakheight_2,Integral_1,Integral_2
0,spectrum2.csv,25.0,5.0,10.0,-50.0
1,spectrum1.csv,25.0,5.0,27.5,-32.5


# Data Exploration

In Data Cleaning you are interested to check the distribution and shape of a variable with missing data.

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

In [ ]:
from ydata_profiling import ProfileReport
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

# Correlation and PPS Analysis

This code has been extracted from the Code Insitute PPS lesson.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps


def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                    linewidth=0.5
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r', annot_kws={"size": font_annot},
                         linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 12), font_annot=8):

    print("\n")
    print("* Analyse how the target variable for your ML models are correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are correlated among themselves")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous variables \n")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(f"PPS detects linear or non-linear relationships between two columns.\n"
          f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

In [ ]:
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman, 
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.4, PPS_Threshold =0.2,
                  figsize=(12,10), font_annot=10)

# Data Cleaning

## Assessing Missing Data Levels

- Custom function to display missing data levels in a DataFrame, it shows the absolute levels, relative levels and data type.=10)

In [ ]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                   "PercentageOfDataset": missing_data_percentage,
                                   "DataType": df.dtypes}
                                    )
                          .sort_values(by=['PercentageOfDataset'], ascending=False)
                          .query("PercentageOfDataset > 0")
                          )

    return df_missing_data

In [ ]:
EvaluateMissingData(df)

- **Enclosed Porch and Wood Deck**: These features will be removed due to a very high percentage of missing data, as they would not provide meaningful information.n

- Missing data in other features will be imputed using appropriate methods to ensure data completeness and integrity.

The data cleaning process aims to improve data quality by addressing missing values, optimizing the dataset, and preparing it for further analysis and modeling.

## Missing data

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):

  flag_count=1 # Indicate plot number
  
  # distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # scan over variables, 
    # first on variables that you applied the method
    # if the variable is a numerical plot, a histogram if categorical plot a barplot
  for set_of_variables in [variables_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:  # it is categorical variable: barplot
        
        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        fig , axes = plt.subplots(figsize=(15, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"])
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: # it is numerical variable: histogram

        fig , axes = plt.subplots(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step", ax=axes)
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step", ax=axes)
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1

### Numerical Values

to inplace the missing values for the missing values it was chosen to impute uisng meanmedianimputer(). Since the median isn't effected by outliers.

In [ ]:
from feature_engine.imputation import MeanMedianImputer

variables_method = ['LotFrontage', 'BedroomAbvGr', '2ndFlrSF', 'GarageYrBlt', 'MasVnrArea']

imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)

df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                  df_cleaned=df_method,
                  variables_applied_with_method=variables_method)

## Categorical values

For categorical features in the dataset, missing values will be addressed using the "Frequent" method. This method replaces missing categorical values with the most frequent category within each respective feature.

Imputing missing values in categorical variables ensures data completeness and prepares the dataset for further analysis and modeling.

In [ ]:
from feature_engine.imputation import CategoricalImputer

variables = ['GarageFinish', 'BsmtFinType1']

imputer = CategoricalImputer(imputation_method='frequent', variables=variables)

df_method = imputer.fit_transform(df)

DataCleaningEffect(df_original=df,
                  df_cleaned=df_method,
                  variables_applied_with_method=variables_method)

## Drop Variables

- dropping the **EnclosedPorch** & **WoodDeckSF features

In [ ]:
from feature_engine.selection import DropFeatures

variables = ['EnclosedPorch', 'WoodDeckSF']

imputer = DropFeatures(features_to_drop=variables)
imputer.fit_transform(df)


## Split Train and Test Set

- The set will be split in train and test set. 20 procent will be used to used to test the model

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

- DropFeatures

In [ ]:
variables_method = ['EnclosedPorch', 'WoodDeckSF']
imputer = DropFeatures(features_to_drop=variables_method)
imputer.fit(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

- MeanMedianImputer

In [ ]:
variables_method = ['LotFrontage', 'BedroomAbvGr', '2ndFlrSF', 'GarageYrBlt', 'MasVnrArea']
imputer = MeanMedianImputer(imputation_method='median', variables=variables_method)

imputer.fit_transform(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

- CategoricalImputer

In [ ]:
variables_method = ['GarageFinish', 'BsmtFinType1']
imputer = CategoricalImputer(imputation_method='missing', fill_value='Unf', variables=variables_method)

imputer.fit_transform(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

- check for missing data

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned') # create outputs/datasets/collection folder
except Exception as e:
  print(e)



In [ ]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)

In [ ]:
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)